# Variational Linear Systems via QAOA

### Abstract

In this notebook, we implement an approach to solving linear systems on a NISQ computer. The approach uses QAOA [1] with the "linear system Hamiltonian" given in [2]. Other approaches for the "quantum linear systems problem" [3-5] require resources beyond current hardware capabilities. This method can be implemented on NISQ processors. Other "near-term approaches" have recently emerged in the literature [6-7].

In [1]:
"""Imports for the notebook.

Requires:
    Numpy
    Scipy
    Sympy
    Cirq v0.5.0
"""
from time import time

import numpy as np
from scipy.linalg import expm, fractional_matrix_power
from scipy.optimize import minimize
from sympy import Symbol

import cirq
assert cirq.__version__ == "0.5.0", "Wrong version of Cirq! Code may or may not work."

## Preliminaries

We seek to solve the "quantum linear systems problem" (QLSP) defined by the system

\begin{equation}
    A \mathbf{x} = \mathbf{b}
\end{equation}

where $A \in \mathbb{R}^{N \times N}$ and $N = 2^n$. QLSP is similar to the classical linear systems problem except we only want to output a "quantum description" of the solution

\begin{equation}
    | \mathbf{x} \rangle = |A^{-1} \mathbf{b} \rangle
\end{equation}

(because we don't have a good way around this).

#### Challenge: Find a (practical) problem where we only want some expectation of the solution $\langle \mathbf{x} | \hat{O} | \mathbf{x} \rangle$ for some Hermitian operator $\hat{O}$.

## Ansatz wavefunction

_Note: This presentation is based off the slides Yigit presented at SQUINT 2019._

We follow the method outlined in [2] to construct our ansatz wavefunction. First, define the projector

\begin{equation}
    P_\mathbf{b}^\perp := I - |\mathbf{b}\rangle \langle \mathbf{b}| .
\end{equation}

Since $P_\mathbf{b}^\perp |\mathbf{b} \rangle = \mathbf{0}$, we have

\begin{equation}
    P_\mathbf{b}^\perp A | \mathbf{x} \rangle = \mathbf{0} .
\end{equation} 

Define $B := P_\mathbf{b}^\perp A$ and the operator

\begin{equation}
    H_f := B^\dagger B = A P_\mathbf{b}^\perp A .
\end{equation} 

The subscript $f$ will become apparent shortly. This operator (Hamiltonian) is:

* Hermitian. 
* positive-semidefinite $\implies$ $|\mathbf{x}\rangle$ is the ground state.

Note that $|\mathbf{x}\rangle$ is the _unique_ ground state since $P_\mathbf{b}^\perp$ is an $N - 1$ dimensional projector.

We now parameterize the linear system $A$ with the schedule

\begin{equation}
    A \mapsto A(t) := (1 - t) I + t A
\end{equation}

for $0 \le t \le 1$ and similarly the Hamiltonian

\begin{equation}
        H(t) := A(t) P_\mathbf{b}^\perp A(t)  .
\end{equation} 

Note that $H(t=1) = H_f$. 

This defines our "driver Hamiltonian" in QAOA, i.e. the unitary operator

\begin{equation}
    U_t(\gamma) = e^{ - i \gamma H(t) }
\end{equation}

#### Question: Normally, the "cost Hamiltonian" in QAOA is assumed to be diagonal in the computational basis. In general, our Hamiltonian is not. Do we need a separate "mixer Hamiltonian" in this case?

#### Note: QAOA typically assumes the driver Hamiltonian is time-independent (so far as I know), but our Hamiltonian is time-dependent. 

We will, for now, take the standard mixer Hamiltonian

\begin{equation}
    V(\beta) = \prod_j e^{ - i \beta X_j} ,
\end{equation}

i.e., a rotation of each qubit about the $x$-axis by $2 \beta$. 

The ansatz wavefunction is thus

\begin{equation}
    |\mathbf{\gamma}, \mathbf{\beta} \rangle := \left[ \prod_{1}^{p} U_{t_i} (\gamma_i) V(\beta_i) \right] H^{\otimes n} |0\rangle^{\otimes n}
\end{equation}

where $n := \log_2 N$. 

# Code Implementation

We now turn to a implementation of this algorithm using Cirq.

## Example Problem

In the cell below we'll generate a random linear system of equations.

In [2]:
"""Generate a system of equations to solve."""
# Seed for the random number generator
SEED = 8675309
np.random.seed(SEED)

# Dimension of system. Assume square for now
n = 2
N = 2**n

# Generate the matrix
A = np.random.rand(N, N)

# Generate the vector
b = np.random.rand(N)

# Do the solution classically
start = time()
x = np.linalg.solve(A, b)
total = time() - start

# Display the system, solution, and time to solve
print("Classically solving the system Ax = b...")
print("A =\n", A)
print("b =\n", b)
print()
print("Solution found in", total, "seconds.")
print("x =\n", x)
print("Ax =\n", np.dot(A, x))

Classically solving the system Ax = b...
A =
 [[0.81245912 0.75104509 0.35736652 0.20229478]
 [0.40900113 0.08212473 0.21498652 0.41388092]
 [0.01816738 0.66977656 0.60010341 0.91928216]
 [0.34272827 0.55517967 0.7175205  0.17863173]]
b =
 [0.92911262 0.76151484 0.12024533 0.63567192]

Solution found in 0.009928703308105469 seconds.
x =
 [ 1.46333144 -0.72895543  0.70855107  0.17045233]
Ax =
 [0.92911262 0.76151484 0.12024533 0.63567192]


## Building up the ansatz

Now we'll build up the ansatz. We'll "cheat" a little by

1. Classically exponentiating the system (to avoid Trotterization).
1. Allowing for matrix gates (to avoid compiling).

We'll use the "half-way point" Hamiltonian. That is, setting $t = 0.5$ in the Hamiltonian $H(t)$. 

In [3]:
"""Get the matrix for the driver Hamiltonian."""
# Set the time for the Hamiltonian
t = 0.5

# Identity matrix, for convenince
iden = np.identity(N)

# Get the "b projector"
Pb = iden - np.outer(b, b)

# Get the parameterized linear system
At = (1.0 - t) * iden + t * A

# Get the Hamiltonian
Ht = At @ Pb @ At.conj().T

# Make sure it's Hermitian
assert np.allclose(Ht, Ht.conj().T)

# Do the matrix exponential
Umat = expm(-1j * Ht)

# Make sure it's unitary
assert np.allclose(Umat.conj().T @ Umat, iden)

This gives us a matrix representation of the $U_t(\gamma)$ unitary -- except without the $\gamma$ parameter. We'll take care of this when we construct a matrix gate in Cirq by raising the gate to the power $\gamma$. We now construct a gate from this unitary that we can use in a circuit.

In [17]:
"""Create a gate for the driver Hamiltonian (unitary)."""
class Driver(cirq.TwoQubitGate):
    """Gate for the U_t(\gamma) driver Hamiltonian (for a particular t)."""
    
    def __init__(self, matrix, gamma):
        self.gamma = gamma
        self.matrix = fractional_matrix_power(matrix, gamma)
    
    def _unitary_(self):
        return self.matrix
    
    def _circuit_diagram_info_(self, args):
        return "Driver^{}".format(self.gamma), "Driver^{}".format(self.gamma)

Now we'll make sure we can implement this in a quantum circuit.

In [18]:
"""Function for building the ansatz circuit."""
def circuit(gamma, beta):
    """Returns the state |gamma, beta> defined above."""
    # Get a qubit register
    qreg = cirq.LineQubit.range(n)

    # Get a quantum circuit
    circ = cirq.Circuit()

    # How about a round of Hadamards?
    circ.append(cirq.H.on_each(*qreg))

    # Do the driver Hamiltonian
    U = Driver(Umat, gamma)
    circ.append(U.on(*qreg))

    # Do the mixer Hamiltonian
    circ.append(cirq.Rx(beta).on_each(*qreg))
    
    return circ

In [19]:
"""Make sure the code above is working properly."""
# Display the circuit
circ = circuit(0.2, 0.5)
print("Circuit:")
print(circ)

# Make sure it can be simulated
sim = cirq.Simulator()
res = sim.simulate(circ)

# Display the final state
print("\nFinal state:")
print("|psi> = ", *res.final_state)

Circuit:
0: ───H───Driver^0.2───Rx(0.159π)───
          │
1: ───H───Driver^0.2───Rx(0.159π)───

Final state:
|psi> =  (0.44485494-0.053723976j) (0.49161464-0.13756692j) (0.4944042-0.24651188j) (0.44841725-0.17979547j)


## Computing the Expectation Value

To compute the expectation value (cost), we use the final Hamiltonian $H_f$:

\begin{equation}
    C(\mathbf{\gamma}, \mathbf{\beta}) = 
        \langle \mathbf{\gamma}, \mathbf{\beta} | H_f | \mathbf{\gamma}, \mathbf{\beta} \rangle
\end{equation}

It is this quantity we minimize during the optimization procedure.

In [20]:
"""Function for computing the cost."""
def cost(gamma, beta, simulator=cirq.Simulator()):
    """Returns the cost C(\gamma, \beta) defined above."""
    # Build the circuit
    circ = circuit(gamma, beta)
    
    # Get the final state
    psi = sim.simulate(circ).final_state
    
    # Return the expectation <\gamma, \beta| Hf | \gamma, \beta>
    Hf = A.conj().T @ Pb @ A
    return abs(np.dot(psi.conj().T, np.dot(Hf, psi)))

## Minimizing the cost

We now minimize the cost. For simplicity, we use built-in optimizers from Scipy.

In [21]:
"""Test the cost at some values."""
# Functional form for scipy.optimize.minimize
def obj(x):
    val = cost(x[0], x[1])
    print("Current cost:", val, end="\r")
    return val

# Do the minimization
start = time()
out = minimize(obj, np.random.rand(2), method="Powell")
total = time() - start

In [22]:
"""See the quantum solution."""
# Get the wavefunction at the best angles
gamma_opt, beta_opt = out["x"]
circ = circuit(gamma_opt, beta_opt)
xquantum = sim.simulate(circ).final_state

# Display the system, solution, and time to solve
print("Quantumly solving the system Ax = b...")
print("A =\n", A)
print("b =\n", b)
print()
print("Solution found in", total, "seconds.")
print("x =\n", x)
print("Ax =\n", np.dot(A, xquantum))
print("Overlap with classical solution:", abs(np.dot(xquantum, x))**2)

Quantumly solving the system Ax = b...
A =
 [[0.81245912 0.75104509 0.35736652 0.20229478]
 [0.40900113 0.08212473 0.21498652 0.41388092]
 [0.01816738 0.66977656 0.60010341 0.91928216]
 [0.34272827 0.55517967 0.7175205  0.17863173]]
b =
 [0.92911262 0.76151484 0.12024533 0.63567192]

Solution found in 0.6759316921234131 seconds.
x =
 [ 1.46333144 -0.72895543  0.70855107  0.17045233]
Ax =
 [0.31580315-0.48984161j 0.1246926 -0.51056664j 0.4222583 -0.95577174j
 0.36942646-0.51631243j]
Overlap with classical solution: 0.6675104483187791


## Higher order QAOA

We now go beyond one layer $p > 1$ in the QAOA ansatz. This is just a slight modification of the code above.

In [25]:
"""Circuit for higher order QAOA."""
def qaoa(gammas, betas):
    """Returns the state |gamma, beta> defined above."""
    # Make sure we have the correct number of parameters
    if len(gammas) != len(betas):
        raise ValueError("gammas and betas must have the same length.")

    # Get a qubit register
    qreg = cirq.LineQubit.range(n)

    # Get a quantum circuit
    circ = cirq.Circuit()

    # How about a round of Hadamards?
    circ.append(cirq.H.on_each(*qreg))
    
    # Do each layer of QAOA
    for gamma, beta in zip(gammas, betas):
        # Do the driver Hamiltonian
        U = Driver(Umat, gamma)
        circ.append(U.on(*qreg))

        # Do the mixer Hamiltonian
        circ.append(cirq.Rx(beta).on_each(*qreg))
    
    return circ

Let's test this to make sure it works properly.

In [26]:
"""Testing the higher order QAOA circuit."""
gammas = [1, 2, 3]
betas = [0.1, 0.2, 0.3]

circ = qaoa(gammas, betas)

print("Circuit:")
print(circ)

Circuit:
0: ───H───Driver^1───Rx(0.032π)───Driver^2───Rx(0.064π)───Driver^3───Rx(0.095π)───
          │                       │                       │
1: ───H───Driver^1───Rx(0.032π)───Driver^2───Rx(0.064π)───Driver^3───Rx(0.095π)───


Now we modify the cost function.

In [29]:
"""Defining the cost function for higher order QAOA."""
def expectation(gammas, betas, simulator=cirq.Simulator()):
    """Returns the cost C(\gamma, \beta) defined above."""
    # Build the circuit
    circ = qaoa(gammas, betas)
    
    # Get the final state
    psi = sim.simulate(circ).final_state
    
    # Return the expectation <\gamma, \beta| Hf | \gamma, \beta>
    Hf = A.conj().T @ Pb @ A
    return abs(np.dot(psi.conj().T, np.dot(Hf, psi)))

And now test it.

In [30]:
"""Testing the cost function for higher order QAOA."""
print(expectation([1, 2], [3, 4]))

0.17538981520367278


And finally minimization for higher order QAOA.

In [31]:
"""Defining the objective function for optimization with multiple layers."""
def objective(x):
    """Returns the objective function for higher order QAOA."""
    # Make sure the number of parameters is valid
    if len(x) % 2 != 0:
        raise ValueError("Invalid number of parameters. Must be even.")
    
    # Parse the parameters (arbitrary convention)
    gammas = x[:len(x) // 2]
    betas = x[len(x) // 2:]
    
    # Compute the expectation for these parameters
    cval = expectation(gammas, betas)
    
    # Print it out
    print("Current cost:", cval, end="\r")
    
    return cval

## Optimizing higher order QAOA

Now we look at performing the optimization.

In [32]:
"""Optimizing higher order QAOA."""
# Do the minimization
start = time()
out = minimize(objective, np.random.rand(2), method="Powell")
total = time() - start

In [ ]:
"""See the quantum solution."""
# Get the wavefunction at the best angles
opt_params = out["x"]
gammas = opt_params[:len(opt_params) // 2]
betas = opt_params[len(opt_params) // 2: ]
circ = circuit(gamma_opt, beta_opt)
xquantum = sim.simulate(circ).final_state

# Display the system, solution, and time to solve
print("Quantumly solving the system Ax = b...")
print("A =\n", A)
print("b =\n", b)
print()
print("Solution found in", total, "seconds.")
print("x =\n", x)
print("Ax =\n", np.dot(A, xquantum))
print("Overlap with classical solution:", abs(np.dot(xquantum, x))**2)

# References

[1] Edward Farhi, Jeffrey Goldstone, Sam Gutmann, A quantum approximate optimization algorithm. https://arxiv.org/abs/1411.4028

[2] Yigit Subasi, Rolando D. Somma, Davide Orsucci, Quantum algorithms for systems of linear equations inspired by adiabatic quantum computing. https://arxiv.org/abs/1805.10549.

[3] HHL

[4] QLSP via QSVE.

[5] QLSP Primer.

[6] NISQLSP I.

[7] NISQLSP II.